<h1>Tratamento da Base</h1>

Os dados foram submetidos a etapa de pré-processamento, visando a seleção das variáveis mais relevantes, tratamento de valores nulos e renomeação de colunas.

In [1]:
# Importações
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
# Colunas que seram importadas
colunas=[
    'DthAtualizaCadastralEmpreend','SigAgente',
    'DscClasseConsumo','DscSubGrupoTarifario',
    'SigUF','NomRegiao',
    'NomMunicipio','SigTipoConsumidor',
    'SigModalidadeEmpreendimento','SigTipoGeracao',
    'DscPorte','MdaPotenciaInstaladaKW'
]

In [3]:
# Carregando o dataset em um dataframe
df = spark.read.csv('empreendimento-geracao-distribuida.csv', header=True, inferSchema=False)\
        .select(colunas)

In [4]:
# Visualizando
df.toPandas().head()

,DthAtualizaCadastralEmpreend,SigAgente,DscClasseConsumo,DscSubGrupoTarifario,SigUF,NomRegiao,NomMunicipio,SigTipoConsumidor,SigModalidadeEmpreendimento,SigTipoGeracao,DscPorte,MdaPotenciaInstaladaKW
0,2015-07-15,ELETROACRE,Comercial,B3,AC,Norte,Rio Branco,PJ,P,UFV,Microgeracao,32
1,2015-04-20,ELETROACRE,Residencial,B1,AC,Norte,Tarauacá,PF,P,UFV,Microgeracao,4
2,2015-06-22,ELETROACRE,Comercial,B3,AC,Norte,Rio Branco,PJ,P,UFV,Microgeracao,2
3,2016-08-25,ELETROACRE,Residencial,B1,AC,Norte,Rio Branco,PF,P,UFV,Microgeracao,2
4,2017-03-09,ELETROACRE,Residencial,B1,AC,Norte,Rio Branco,PF,P,UFV,Microgeracao,5


In [5]:
# Renomeando colunas
nome_coluna = [
    'Data','Agente','ClasseConsumo','GrupoTarifario','UF','Regiao','Municipio',
    'Consumidor','ModalidadeEmpreend','TipoGeracao','Porte','PotenciaKW'
]

for indice, coluna in enumerate(df.columns):
    df = df.withColumnRenamed(coluna , nome_coluna[indice])

In [6]:
# verificando o tipo de dados das colunas
df.printSchema()

root
 |-- Data: string (nullable = true)
 |-- Agente: string (nullable = true)
 |-- ClasseConsumo: string (nullable = true)
 |-- GrupoTarifario: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- Regiao: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Consumidor: string (nullable = true)
 |-- ModalidadeEmpreend: string (nullable = true)
 |-- TipoGeracao: string (nullable = true)
 |-- Porte: string (nullable = true)
 |-- PotenciaKW: string (nullable = true)



In [7]:
# Transformando os dados da coluna data no tipo Timestamp
df = df.withColumn('Data', col('Data').cast(TimestampType()))

In [8]:
# Conferindo mudança
df.printSchema()

root
 |-- Data: timestamp (nullable = true)
 |-- Agente: string (nullable = true)
 |-- ClasseConsumo: string (nullable = true)
 |-- GrupoTarifario: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- Regiao: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Consumidor: string (nullable = true)
 |-- ModalidadeEmpreend: string (nullable = true)
 |-- TipoGeracao: string (nullable = true)
 |-- Porte: string (nullable = true)
 |-- PotenciaKW: string (nullable = true)



In [9]:
# Consulta se há valores nulos
for coluna in df.columns:
    print(coluna, df.filter(col(coluna).isNull()).count())

Data 551
Agente 70
ClasseConsumo 0
GrupoTarifario 0
UF 31
Regiao 0
Municipio 31
Consumidor 2
ModalidadeEmpreend 2
TipoGeracao 0
Porte 0
PotenciaKW 12595


In [11]:
# Excluindo valore nulos
df_tratado = df.toPandas().dropna()

In [12]:
# Verificando se os nulos foram excluidos
df_tratado.isnull().sum()

Data                  0
Agente                0
ClasseConsumo         0
GrupoTarifario        0
UF                    0
Regiao                0
Municipio             0
Consumidor            0
ModalidadeEmpreend    0
TipoGeracao           0
Porte                 0
PotenciaKW            0
dtype: int64

In [13]:
# Visualizando dimensões do dataframe
print('Linhas:',df_tratado.shape[0])
print('Colunas:',df_tratado.shape[1])

Linhas: 951262
Colunas: 12


In [14]:
# Visualizando dataframe tratado
df_tratado

,Data,Agente,ClasseConsumo,GrupoTarifario,UF,Regiao,Municipio,Consumidor,ModalidadeEmpreend,TipoGeracao,Porte,PotenciaKW
0,2015-07-15,ELETROACRE,Comercial,B3,AC,Norte,Rio Branco,PJ,P,UFV,Microgeracao,32
1,2015-04-20,ELETROACRE,Residencial,B1,AC,Norte,Tarauacá,PF,P,UFV,Microgeracao,4
2,2015-06-22,ELETROACRE,Comercial,B3,AC,Norte,Rio Branco,PJ,P,UFV,Microgeracao,2
3,2016-08-25,ELETROACRE,Residencial,B1,AC,Norte,Rio Branco,PF,P,UFV,Microgeracao,2
4,2017-03-09,ELETROACRE,Residencial,B1,AC,Norte,Rio Branco,PF,P,UFV,Microgeracao,5
...,...,...,...,...,...,...,...,...,...,...,...,...
964506,2022-04-14,CEMIG-D,Residencial,B1,TO,Norte,Oliveira de Fátima,PF,P,UFV,Microgeracao,5
964507,2022-04-14,CEMIG-D,Residencial,B1,TO,Norte,Oliveira de Fátima,PF,P,UFV,Microgeracao,3
964508,2022-04-13,CEMIG-D,Residencial,B1,TO,Norte,Divinópolis do Tocantins,PF,R,UFV,Microgeracao,3
964509,2022-04-28,CEMIG-D,Residencial,B1,TO,Norte,Divinópolis do Tocantins,PF,P,UFV,Microgeracao,6


In [15]:
# Salvando o datafram em um dataset
df_tratado.to_csv('empreendimento-geracao-distribuida-tratado.csv', index=False)